In [1]:
import json
import os
import numpy as np
from tqdm import tqdm
import shutil
import cv2
from xml.etree import cElementTree as ET
from glob import glob

In [18]:
def extract_voc(xmlp, default_class_name=None):
    if not os.path.exists(xmlp):
        print('{} file is not exists.'.format(xmlp))
        return 
    with open(xmlp) as f:
        tree = ET.parse(f)
        root = tree.getroot()
    lines = []
    objects = root.findall('object')
    if len(objects) >0:
        for obj in objects:
            name = obj.find('name').text #class name
            if default_class_name is not None:
                name = default_class_name
            bnd = obj.find('bndbox')
            xmin = bnd.find('xmin').text
            ymin = bnd.find('ymin').text
            xmax = bnd.find('xmax').text
            ymax = bnd.find('ymax').text
            line = name + '\t' + ','.join(map(str, [xmin, ymin, xmax, ymax]))
            lines.append(line)
        return lines
    return 

# Truncates numbers to N decimals
def truncate(n, decimals=0):
    multiplier = 10 ** decimals
    return int(n * multiplier) / multiplier

def get_dataset_categorys(xml_paths):
    categorys = []
    for xmlp in xml_paths:
        voc = extract_voc(xmlp)
        if voc is not None:
            for line in voc:
                name, loc = line.split('\t')
                categorys.append(name)
    return list(set(categorys))

def yolo_format(line, imgH, imgW):
    name, loc = line.split('\t')
    xmin, ymin, xmax, ymax = map(int, loc.split(','))
    dw, dh = 1. / imgW, 1 / imgH
    x = (xmin + xmax)/2
    y = (ymin + ymax)/2
    w = xmax - xmin
    h = ymax - ymin
#     print('x:', x, ' ', 'y:', y, ' ', 'w:', dw, 'h:', dh, 'imgW:', imgW, ' ', 'imgH:', imgH)
    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    new_line = ' '.join([name, 
                 str(truncate(x, 7)), 
                 str(truncate(y, 7)), 
                 str(truncate(w, 7)), 
                 str(truncate(h, 7))])
        
    return new_line

def voc2yolo(xml_paths, class_idxs, img_store, txt_store, error, default_class_name=None, exclude_class=[]):
    if not os.path.exists(img_store):
        os.makedirs(img_store)
    if not os.path.exists(txt_store):
        os.makedirs(txt_store)
        
    for xmlp in tqdm(xml_paths):
#         try:
        imgp = xmlp.replace('xml', 'jpg')
        img = cv2.imread(imgp)
        if img is None:
            print(xmlp)
            print(imgp)
            continue
        imgH, imgW = img.shape[:2]
        name = os.path.basename(xmlp)
        txt_store_path = os.path.join(txt_store, name.replace('xml', 'txt')) #txt file store path
        img_store_path = os.path.join(img_store, name.replace('xml', 'jpg')) #image file store path
        voc = extract_voc(xmlp, default_class_name)
        new_lines = []
        if voc is not None:
            for line in voc:

                name, loc = line.split('\t')
                loc = [str(int(lc)) for lc in map(float, loc.split(','))]
                loc = ','.join(loc)
                if name in exclude_class:
                    continue
                idx = class_idxs.get(name, None)
                if idx is None:
                    continue
                line = str(idx) + '\t' + loc
                new_line = yolo_format(line, imgH, imgW)
                new_lines.append(new_line)
            if len(new_lines) >0:
                with open(txt_store_path, 'w') as f:
                    f.write('\n'.join(new_lines))
                cv2.imwrite(img_store_path, img)
#             else:
#                 shutil.move(xmlp, error)
#                 shutil.move(imgp, error)
#         else:
#             shutil.move(xmlp, error)
#             shutil.move(imgp, error)
#         except:
#                 shutil.move(xmlp, error)
#                 shutil.move(imgp, error)
#                 continue

In [6]:
# class_names = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
#         'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
#         'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
#         'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard',
#         'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
#         'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
#         'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 
#         'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 
#         'teddy bear', 'hair drier', 'toothbrush', 'shop']
# class_names = ['bicycle','car', 'motor', 'bus', 'truck', 'trashbin', 'tricycle', 'shop']
# class_names = ['motor', 'trashbin', 'tricycle', 'shop']
class_names = ['construction_trash_dump','domestic_trash_dump',
               'domestic_trash','trash_can_uncovered',
               'trash_can_overflow','construction_trash',
               'trash_can_fall','trash_can']
class_idxs = {name:idx for idx, name in enumerate(class_names)}

In [4]:
xml_paths = glob('../chengguan/trash_can/pachong/org_anno/*.xml')
get_dataset_categorys(xml_paths)

['battery_car',
 'motorcycle',
 'construction_trash_dump',
 'bus',
 'mobile_stall',
 'domestic_trash_dump',
 'domestic_trash',
 'trash_can_uncovered',
 'trash_can_overflow',
 'bulldozer',
 'car',
 'cow',
 'dog',
 'construction_trash',
 'dredger',
 'trash_can_fall',
 'tricycle',
 'truck',
 'fixed_stall',
 'bear',
 'trash_can']

In [19]:
xml_paths = glob('../chengguan/trash_can/pachong/org/*.xml')

img_store = '../chengguan/trash_can/pachong/images/train'
txt_store = '../chengguan/trash_can/pachong/labels/train'
error = '../chengguan/trash_can/pachong/error/'

voc2yolo(xml_paths, class_idxs, img_store, txt_store, error, exclude_class=['battery_car',
 'motorcycle',
 'bus',
 'mobile_stall',
 'bulldozer',
 'car',
 'cow',
 'dog',
 'dredger',
 'tricycle',
 'truck',
 'fixed_stall',
 'bear'])

 25%|██▌       | 1462/5805 [00:05<00:16, 268.57it/s]

../chengguan/trash_can/pachong/org/67.xml
../chengguan/trash_can/pachong/org/67.jpg


 59%|█████▉    | 3428/5805 [00:13<00:09, 240.45it/s]

../chengguan/trash_can/pachong/org/917.xml
../chengguan/trash_can/pachong/org/917.jpg


 88%|████████▊ | 5087/5805 [00:21<00:03, 217.89it/s]

../chengguan/trash_can/pachong/org/810.xml
../chengguan/trash_can/pachong/org/810.jpg


100%|██████████| 5805/5805 [00:24<00:00, 238.40it/s]


In [6]:
# xml_paths = glob('/mnt/data/street/trashbin/*.xml') + \
#             glob('/mnt/data/street/motor/*.xml') + \
#             glob('/mnt/data/street/tricycle/*.xml') + \
#             glob('/mnt/data/street/shop/*.xml') + \
#             glob('/mnt/data/street/true_camera/*.xml')

# img_store = '/mnt/data/street_rz_4class/train/images'
# txt_store = '/mnt/data/street_rz_4class/train/labels'
# error = '/mnt/data/street/error/'

# voc2yolo(xml_paths, class_idxs, img_store, txt_store, error, exclude_class=['bicycle','car', 'bus', 'truck'])

100%|██████████| 3190/3190 [00:25<00:00, 126.19it/s]
